<a href="https://colab.research.google.com/github/sam134-eng/INM705/blob/main/LSTM_Sentiment_GloVe_Integrated%20test%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# --- Ensure NLTK data is downloaded ---
# Calling nltk.download() directly without checking for existence first,
# as the downloader itself handles whether the resource is already present.
# Setting quiet=False to show download progress.
print("Ensuring 'punkt' NLTK resource is downloaded...")
nltk.download('punkt', quiet=False)
print("Ensuring 'stopwords' NLTK resource is downloaded...")
nltk.download('stopwords', quiet=False)
# Add the download for 'punkt_tab' as suggested by the error
print("Ensuring 'punkt_tab' NLTK resource is downloaded...")
nltk.download('punkt_tab', quiet=False)


# --- 1. Load and Preprocess Data ---
# Assuming 'Tweets.csv' is available in the environment.
# In a Colab environment, you would typically upload it first.
# For this self-contained code, we'll assume it's loaded.
# If running locally, ensure 'Tweets.csv' is in the same directory.
try:
    df = pd.read_csv("Tweets.csv")
except FileNotFoundError:
    print("Error: 'Tweets.csv' not found. Please ensure the file is uploaded or in the correct directory.")
    # Create a dummy DataFrame for demonstration if file is not found
    data = {
        'text': [
            "@VirginAmerica What @dhepburn said.",
            "@VirginAmerica plus you've added commercials to the experience... #fail",
            "@VirginAmerica I didn't today... Must mean I need to fly it again!",
            "@VirginAmerica it's really aggressive to blast obnoxious "
            "loud commercials into little earbuds. #anditshardtohearanyone",
            "@VirginAmerica and it's a really big bad thing about it"
        ],
        'airline_sentiment': ['neutral', 'negative', 'positive', 'negative', 'negative']
    }
    df = pd.DataFrame(data)
    print("Using a dummy DataFrame for demonstration.")


# Keep only relevant columns: 'text' and 'airline_sentiment'
df = df[['text', 'airline_sentiment']]

# Filter to ensure only 'positive', 'neutral', 'negative' sentiments are included
df = df[df['airline_sentiment'].isin(['positive', 'neutral', 'negative'])]

# Map sentiment labels to integers for model training
label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
df['label'] = df['airline_sentiment'].map(label_map)

# Define stopwords for text cleaning
stop_words = set(stopwords.words('english'))

# Function to clean and tokenize text
def clean_text(text):
    """
    Cleans the input text by:
    - Removing URLs, mentions (@), hashtags (#)
    - Removing non-alphabetic characters
    - Converting to lowercase
    - Tokenizing the text
    - Removing stopwords
    """
    text = re.sub(r"http\S+|@\w+|#\w+|[^a-zA-Z\s]", "", text.lower())
    # word_tokenize relies on the 'punkt' tokenizer data
    tokens = word_tokenize(text)
    return " ".join([t for t in tokens if t not in stop_words]) # Join tokens back to string for TF-IDF

# Apply the cleaning function to the 'text' column
df['clean_text'] = df['text'].apply(clean_text)

# --- 2. Prepare Data for TF-IDF Vectorization ---
X = df['clean_text']
y = df['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Initialize TF-IDF Vectorizer
# max_features: limits the number of features (vocabulary size)
vectorizer = TfidfVectorizer(max_features=5000)

# Fit the vectorizer on training data and transform both training and test data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Convert sparse TF-IDF matrices to dense NumPy arrays, then to PyTorch Tensors
# PyTorch's nn.Linear layer typically expects dense inputs.
X_train_tensor = torch.tensor(X_train_tfidf.toarray(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_tfidf.toarray(), dtype=torch.float32)

# Convert labels to PyTorch LongTensors (required for nn.CrossEntropyLoss)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create TensorDatasets and DataLoaders for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64 # Define batch size for training
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# --- 3. Define PyTorch Logistic Regression Model ---
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(LogisticRegressionModel, self).__init__()
        # Define a single linear layer for logistic regression
        # input_dim: number of features from TF-IDF (max_features)
        # num_classes: number of sentiment categories (3: negative, neutral, positive)
        self.linear = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        # Forward pass: apply the linear transformation
        # nn.CrossEntropyLoss will apply softmax internally, so no explicit softmax here
        return self.linear(x)

# Get input dimension (number of features from TF-IDF)
input_dim = X_train_tensor.shape[1]
num_classes = len(label_map) # Number of unique sentiment classes

# Instantiate the model
model = LogisticRegressionModel(input_dim, num_classes)

# --- 4. Define Loss Function and Optimizer ---
# CrossEntropyLoss is suitable for multi-class classification
# It combines LogSoftmax and NLLLoss (Negative Log Likelihood Loss)
criterion = nn.CrossEntropyLoss()

# Adam optimizer is a good general-purpose optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001) # Learning rate can be tuned

# --- 5. Train the Model ---
num_epochs = 10 # Number of training epochs

print("\n--- Starting PyTorch Model Training ---")
for epoch in range(num_epochs):
    model.train() # Set the model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

print("\n--- Training Complete ---")

# --- 6. Evaluate the Model ---
model.eval() # Set the model to evaluation mode
all_predictions = []
all_labels = []

with torch.no_grad(): # Disable gradient calculation for evaluation
    for inputs, labels in test_loader:
        outputs = model(inputs)
        # Get the predicted class (index of the max log-probability)
        _, predicted = torch.max(outputs.data, 1)

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Generate classification report
print("\nTF-IDF + PyTorch Logistic Regression Performance:")
print(classification_report(all_labels, all_predictions, target_names=['Negative', 'Neutral', 'Positive']))

In [ ]:
# prompt: generate accuracy/loss plots
# generate classification report
#  generate confusion matrix

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# Generate confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)
print("\nConfusion Matrix:")
print(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Greens', xticklabels=['Negative', 'Neutral', 'Positive'], yticklabels=['Negative', 'Neutral', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

embedding_dim = 100
glove_path = "glove.6B.100d.txt"

glove_embeddings = {}
with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        word = parts[0]
        vector = np.array(parts[1:], dtype='float32')
        glove_embeddings[word] = vector

embedding_matrix = np.zeros((len(vocab), embedding_dim))
for word, idx in vocab.items():
    vector = glove_embeddings.get(word)
    if vector is not None:
        embedding_matrix[idx] = vector


In [ ]:
# --- 7. Implement and Train an LSTM Model ---

# This section implements a basic LSTM model for sentiment classification
# and prepares the data differently than the TF-IDF approach.

print("\n--- Starting LSTM Model Implementation and Training ---")

# --- 7.1 Data Preparation for LSTM ---
# Requires tokenization, vocabulary creation, integer encoding, and padding

# Ensure NLTK data is downloaded (already done at the beginning, but good practice to check)
# nltk.download('punkt', quiet=False) # Already called earlier
# nltk.download('stopwords', quiet=False) # Already called earlier

# Use the clean_text function defined earlier for initial cleaning
# We need to re-tokenize the cleaned text into lists of words for sequencing
df['clean_tokens'] = df['clean_text'].apply(word_tokenize)

# Build a vocabulary
from collections import Counter

# Flatten the list of lists into a single list of words
all_words = [word for tokens in df['clean_tokens'] for word in tokens]
# Count word frequencies
word_counts = Counter(all_words)
# Create a vocabulary, including special tokens for padding (<pad>) and unknown words (<unk>)
# Limit vocabulary size to manage complexity
vocab_size = 10000 # Example vocabulary size
# Get the most common words
most_common_words = word_counts.most_common(vocab_size - 2) # Leave space for <pad> and <unk>
vocab = {word: i + 2 for i, (word, count) in enumerate(most_common_words)}
vocab['<pad>'] = 0
vocab['<unk>'] = 1

# Integer encode the tokens
def encode_tokens(tokens, vocab):
    return [vocab.get(token, vocab['<unk>']) for token in tokens]

df['encoded_text'] = df['clean_tokens'].apply(lambda tokens: encode_tokens(tokens, vocab))

# Pad sequences to a fixed length
max_seq_length = 100 # Example maximum sequence length

def pad_sequence(sequence, max_length, padding_value=0):
    if len(sequence) > max_length:
        return sequence[:max_length] # Truncate if too long
    else:
        return sequence + [padding_value] * (max_length - len(sequence)) # Pad if too short

df['padded_encoded_text'] = df['encoded_text'].apply(lambda seq: pad_sequence(seq, max_seq_length, vocab['<pad>']))

# Prepare data for PyTorch DataLoader
X_lstm = torch.tensor(df['padded_encoded_text'].tolist(), dtype=torch.long) # Use LongTensor for embeddings
y_lstm = torch.tensor(df['label'].values, dtype=torch.long)

# Split data for LSTM (using the same split as TF-IDF for fair comparison, but re-preparing tensors)
X_lstm_train, X_lstm_test, y_lstm_train, y_lstm_test = train_test_split(
    X_lstm, y_lstm, stratify=y_lstm, test_size=0.2, random_state=42
)

# Create TensorDatasets and DataLoaders for LSTM
lstm_batch_size = 64 # Can tune this
lstm_train_dataset = TensorDataset(X_lstm_train, y_lstm_train)
lstm_test_dataset = TensorDataset(X_lstm_test, y_lstm_test)

lstm_train_loader = DataLoader(lstm_train_dataset, batch_size=lstm_batch_size, shuffle=True)
lstm_test_loader = DataLoader(lstm_test_dataset, batch_size=lstm_batch_size, shuffle=False)


# --- 7.2 Define PyTorch LSTM Model ---

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes, num_layers=1, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=vocab['<pad>'])
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        # Use batch_first=True so input tensors are (batch_size, seq_length, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, num_classes) # Final dense layer

    def forward(self, x):
        # x has shape (batch_size, seq_length) - indices
        embedded = self.embedding(x) # Shape: (batch_size, seq_length, embedding_dim)
        # LSTM returns output and (hidden_state, cell_state)
        lstm_out, (hidden, cell) = self.lstm(embedded)
        # We'll use the hidden state from the last time step of the last layer
        # hidden shape: (num_layers * num_directions, batch_size, hidden_dim)
        # Assuming num_directions is 1 (default), we take hidden[-1, :, :]
        # If bidirectional, it would be hidden[-2:, :, :].mean(0) or similar.
        # For simplicity with num_layers=1 and unidirectional:
        hidden_last_layer = hidden[-1, :, :] # Shape: (batch_size, hidden_dim)
        output = self.dropout(hidden_last_layer)
        output = self.fc(output) # Shape: (batch_size, num_classes)
        return output

# --- 7.3 Instantiate and Train the LSTM Model ---

# Hyperparameters for LSTM (can be tuned)
embedding_dim = 100 # Dimension of word embeddings
hidden_dim = 128    # Number of features in the hidden state of the LSTM
lstm_num_layers = 1 # Number of LSTM layers
lstm_dropout = 0.5  # Dropout rate
lstm_learning_rate = 0.001
lstm_num_epochs = 10 # Example number of epochs

lstm_model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim, num_classes, num_layers=lstm_num_layers, dropout=lstm_dropout)

# Define loss function and optimizer for LSTM
lstm_criterion = nn.CrossEntropyLoss()
lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=lstm_learning_rate)

print("\n--- Training LSTM Model ---")
lstm_model.train()
for epoch in range(lstm_num_epochs):
    running_loss = 0.0
    for inputs, labels in lstm_train_loader:
        lstm_optimizer.zero_grad()
        outputs = lstm_model(inputs)
        loss = lstm_criterion(outputs, labels)
        loss.backward()
        lstm_optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(lstm_train_dataset)
    print(f"LSTM Epoch [{epoch+1}/{lstm_num_epochs}], Loss: {epoch_loss:.4f}")

print("\n--- LSTM Training Complete ---")


# --- 7.4 Evaluate the LSTM Model ---

print("\n--- Evaluating LSTM Model ---")
lstm_model.eval()
lstm_all_predictions = []
lstm_all_labels = []

with torch.no_grad():
    for inputs, labels in lstm_test_loader:
        outputs = lstm_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        lstm_all_predictions.extend(predicted.cpu().numpy())
        lstm_all_labels.extend(labels.cpu().numpy())

# Generate classification report for LSTM
print("\nLSTM Performance:")
print(classification_report(lstm_all_labels, lstm_all_predictions, target_names=['Negative', 'Neutral', 'Positive']))

# Generate confusion matrix for LSTM
lstm_conf_matrix = confusion_matrix(lstm_all_labels, lstm_all_predictions)
print("\nLSTM Confusion Matrix:")
print(lstm_conf_matrix)

# Plot confusion matrix for LSTM
plt.figure(figsize=(8, 6))
sns.heatmap(lstm_conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Neutral', 'Positive'], yticklabels=['Negative', 'Neutral', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('LSTM Confusion Matrix')
plt.show()


# --- 8. Final Comparison Summary ---
# Now you have the results for both Logistic Regression (from the previous section's final evaluation)
# and LSTM. You can manually compare the classification reports and confusion matrices.

print("\n--- Final Performance Comparison Summary ---")
print("Metric       | Logistic Regression (Tuned) | LSTM")
print("-----------------|-----------------------------|------")
# Placeholder values - replace with the actual metrics from the reports above
lr_report = classification_report(final_labels, final_predictions, output_dict=True)
lstm_report = classification_report(lstm_all_labels, lstm_all_predictions, output_dict=True)

print(f"Accuracy     | {lr_report['accuracy']:.4f}               | {lstm_report['accuracy']:.4f}")
print(f"Precision (Neg)| {lr_report['0']['precision']:.4f}          | {lstm_report['0']['precision']:.4f}")
print(f"Recall (Neg) | {lr_report['0']['recall']:.4f}             | {lstm_report['0']['recall']:.4f}")
print(f"F1-Score (Neg)| {lr_report['0']['f1-score']:.4f}                 | {lstm_report['0']['f1-score']:.4f}")
print(f"Precision (Neu)| {lr_report['1']['precision']:.4f}          | {lstm_report['1']['precision']:.4f}")
print(f"Recall (Neu) | {lr_report['1']['recall']:.4f}             | {lstm_report['1']['recall']:.4f}")
print(f"F1-Score (Neu)| {lr_report['1']['f1-score']:.4f}                 | {lstm_report['1']['f1-score']:.4f}")
print(f"Precision (Pos)| {lr_report['2']['precision']:.4f}          | {lstm_report['2']['precision']:.4f}")
print(f"Recall (Pos) | {lr_report['2']['recall']:.4f}             | {lstm_report['2']['recall']:.4f}")
print(f"F1-Score (Pos)| {lr_report['2']['f1-score']:.4f}                 | {lstm_report['2']['f1-score']:.4f}")

In [ ]:

# The variables `final_labels` and `final_predictions` were used in the summary
# but were not defined in the TF-IDF Logistic Regression evaluation section.
# They should be replaced with `all_labels` and `all_predictions` respectively.

# --- 8. Final Comparison Summary ---
# Now you have the results for both Logistic Regression (from the previous section's final evaluation)
# and LSTM. You can manually compare the classification reports and confusion matrices.

print("\n--- Final Performance Comparison Summary ---")
print("Metric         | Logistic Regression | LSTM")
print("-----------------|---------------------|------")
# Use the actual variables from the TF-IDF evaluation
lr_report = classification_report(all_labels, all_predictions, output_dict=True)
lstm_report = classification_report(lstm_all_labels, lstm_all_predictions, output_dict=True)

print(f"Accuracy       | {lr_report['accuracy']:.4f}         | {lstm_report['accuracy']:.4f}")
# Access individual class metrics safely, handling potential missing keys (though unlikely here)
neg_lr = lr_report.get('0', {})
neu_lr = lr_report.get('1', {})
pos_lr = lr_report.get('2', {})

neg_lstm = lstm_report.get('0', {})
neu_lstm = lstm_report.get('1', {})
pos_lstm = lstm_report.get('2', {})


print(f"Precision (Neg)| {neg_lr.get('precision', 0.0):.4f}         | {neg_lstm.get('precision', 0.0):.4f}")
print(f"Recall (Neg)   | {neg_lr.get('recall', 0.0):.4f}         | {neg_lstm.get('recall', 0.0):.4f}")
print(f"F1-Score (Neg) | {neg_lr.get('f1-score', 0.0):.4f}         | {neg_lstm.get('f1-score', 0.0):.4f}")

print(f"Precision (Neu)| {neu_lr.get('precision', 0.0):.4f}         | {neu_lstm.get('precision', 0.0):.4f}")
print(f"Recall (Neu)   | {neu_lr.get('recall', 0.0):.4f}         | {neu_lstm.get('recall', 0.0):.4f}")
print(f"F1-Score (Neu) | {neu_lr.get('f1-score', 0.0):.4f}         | {neu_lstm.get('f1-score', 0.0):.4f}")

print(f"Precision (Pos)| {pos_lr.get('precision', 0.0):.4f}         | {pos_lstm.get('precision', 0.0):.4f}")
print(f"Recall (Pos)   | {pos_lr.get('recall', 0.0):.4f}         | {pos_lstm.get('recall', 0.0):.4f}")
print(f"F1-Score (Pos) | {pos_lr.get('f1-score', 0.0):.4f}         | {pos_lstm.get('f1-score', 0.0):.4f}")

